# $M^6 - (GPT)^3$ Composer

### Imports

In [1]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import json
import os
from datetime import datetime
from generation.chat import Chat

### Utils

In [2]:
# Widgets
chat_display = widgets.HTML(value="", placeholder="Chat will be displayed here...")
message_input = widgets.Text(placeholder='Enter your message...')
send_button = widgets.Button(description='Send')
json_edit_area = widgets.Textarea(value='', placeholder='Edit JSON here...', layout=widgets.Layout(width='100%', height='300px'))
save_button = widgets.Button(description='Save JSON')
update_context_button = widgets.Button(description='Update Context')

def add_message_to_chat(sender, message):
    current_chat = chat_display.value
    formatted_message = f"<b>{sender}:</b> {message}<br>"
    chat_display.value = current_chat + formatted_message


def send_message(_):
    with output_area:
        clear_output() 
        user_message = message_input.value
        if user_message:
            add_message_to_chat("You", user_message)
            message_input.value = ""
            response_json = chat.send_user_message(user_message, model, temperature)
            response_json = response_json.replace("```json", "").replace("```", "").strip()
            try:
                response = json.loads(response_json)
            except json.JSONDecodeError as e:
                print(f"JSON load error. Response of the Chat is probably unterminated: {e}")
            add_message_to_chat("Chat", response['com'])
            json_edit_area.value = json.dumps(response, indent=4)


def update_context_from_json(_):
    edited_json = json_edit_area.value
    try:
        response = json.loads(edited_json)
        
        # Update context
        if 'name' in response:
            user_message = f"Write me a {response['name']}"
            chat.context.append({'role': 'user', 'content': user_message})
        
        assistant_response = json.dumps(response, indent=4)
        chat.context.append({'role': 'assistant', 'content': assistant_response})

        with output_area:
            clear_output()
            print("Context updated successfully.")
    except json.JSONDecodeError as e:
        with output_area:
            clear_output()
            print(f"JSON decode error: {e}")

def save_json(_):
    edited_json = json_edit_area.value
    try:
        response = json.loads(edited_json)
    except json.JSONDecodeError as e:
        with output_area:
            clear_output()
            print(f"JSON decode error: {e}")
        return

    # Save JSON to a file
    song_name = response.get("name", "untitled")
    current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
    file_name = f'{song_name}_{current_time}.json'
    output_dir = "./generated_json"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    full_path = os.path.join(output_dir, file_name)
    with open(full_path, 'w') as file:
        json.dump(response, file, indent=4)

    chat.context = response['context']

    with output_area:
        clear_output()
        print(f"Response JSON saved to {full_path}")

### Api key

In [3]:
# LLM parameters
api_key = open("OPENAI_API_KEY.txt", "r").read().strip("\n") # key for OpenAI API (might be placed in file or explicitly as string)
model = "gpt-3.5-turbo"
temperature = 0.7

chat = Chat(api_key)

In [4]:
# Display Chat and JSON editor
output_area = widgets.Output()
update_context_button.on_click(update_context_from_json)
send_button.on_click(send_message)
save_button.on_click(save_json)
display(chat_display, message_input, send_button, json_edit_area, update_context_button, save_button, output_area)

HTML(value='', placeholder='Chat will be displayed here...')

Text(value='', placeholder='Enter your message...')

Button(description='Send', style=ButtonStyle())

Textarea(value='', layout=Layout(height='300px', width='100%'), placeholder='Edit JSON here...')

Button(description='Update Context', style=ButtonStyle())

Button(description='Save JSON', style=ButtonStyle())

Output()

In [5]:
edited_json = json_edit_area.value
try:
    response = json.loads(edited_json)
except json.JSONDecodeError as e:
    with output_area:
        clear_output()
        print(f"JSON decode error: {e}")

In [6]:
from generation.generate_midi import MidiGenerator
midi_generator = MidiGenerator()

In [8]:
# Generate song from JSON
generation = 100
population = 128
midi_filename = midi_generator.generate_midi_from_json(response, generation, population)
print(f"MIDI file generated: {midi_filename}")

JSON Structure:
[{'s': 'Verse', 'val': 0.5, 'ar': 0.8}, {'s': 'Chorus', 'val': 0.7, 'ar': 0.9}, {'s': 'Verse', 'val': 0.6, 'ar': 0.7}, {'s': 'Chorus', 'val': 0.8, 'ar': 1}]
Chat: Creating a journey through contrasting sections with varied scales and instrumentation to evoke a sense of exploration and wonder.
{'c': 'D', 'dur': 2}
{'c': 'G', 'dur': 1}
{'c': 'A', 'dur': 1}
{'c': 'D', 'dur': 2}
{'c': 'G', 'dur': 1}
{'c': 'A', 'dur': 1}
Generating bass for: synth
Generation 1/10
Generation 2/10
Generation 3/10
Generation 4/10
Generation 5/10
Generation 6/10
Generation 7/10
Generation 8/10
Generation 9/10
Generation 10/10
Generating motif for: violin
Generation 1/10
Generation 2/10
Generation 3/10
Generation 4/10
Generation 5/10
Generation 6/10
Generation 7/10
Generation 8/10
Generation 9/10
Generation 10/10
Generating melody for: flute
Generation 1/10
Generation 2/10
Generation 3/10
Generation 4/10
Generation 5/10
Generation 6/10
Generation 7/10
Generation 8/10
Generation 9/10
Generation 10

In [9]:
# Generate random song (no LLM)
generation = 100
population = 128
midi_filename = midi_generator.generate_midi_random(generation, population)
print(f"MIDI file generated: {midi_filename}")

JSON Structure:
[{'s': 'Intro', 'val': 0.0, 'ar': 0.5}]
Valence: 0.866186373727472
Arousal: 0.18352228370176604
Scale: C Major
Tempo: 93
Meter: (14, 8)
Bass Sound: None, Bass Technique: None
Motif Sound: marimba, Motif Technique: long_motif
Perc Sound: standard, Perc Technique: full
Melody Sound: None, Melody Technique: None
Dm
F
G
Am
C
Dm
G
G
Bdim
Am
C
C
Am
Dm
Dm
Bdim
F
F
Bdim
Em
Generating motif for: marimba
Generation 1/10
Generation 2/10
Generation 3/10
Generation 4/10
Generation 5/10
Generation 6/10
Generation 7/10
Generation 8/10
Generation 9/10
Generation 10/10
MIDI file generated: .\generated_midi\Randomized_20240820_001116.mid
